In [1]:
'''
CS585_Lab3
CS585 Image and Video Computing
Lab 3
--------------
This program introduces the following concepts:
	a) Reading a stream of images from a webcamera, and displaying the video
	b) Skin color detection
	c) Background differencing
	d) Visualizing motion history
--------------
'''
import cv2
import sys
import numpy as np

In [2]:
def my_skin_detect(src):
    '''
    Start with red > 95 and go from there
    
    
    Function that detects whether a pixel belongs to the skin based on RGB values
    Args: 
        src The source color image
    Returns: 
        dst The destination grayscale image where skin pixels are colored white and the rest are colored black
    Surveys of skin color modeling and detection techniques:
    Vezhnevets, Vladimir, Vassili Sazonov, and Alla Andreeva. "A survey on pixel-based skin color detection techniques." Proc. Graphicon. Vol. 3. 2003.
    Kakumanu, Praveen, Sokratis Makrogiannis, and Nikolaos Bourbakis. "A survey of skin-color modeling and detection methods." Pattern recognition 40.3 (2007): 1106-1122.
    '''
    dst = np.zeros(np.shape(src)[:-1], dtype=np.uint8)
    
    mask = np.logical_and.reduce((src[:,:,0] > 20, src[:,:,1] > 40, src[:,:,2] > 95, 
                                    src.max(axis=-1) - src.min(axis=-1) > 15, 
                                    abs(src[:,:,2] - src[:,:,1]) > 15, 
                                    src[:,:,2] > src[:,:,1], src[:,:,2] > src[:,:,0]))
    
    dst[mask] = 255

    # Too slow
    # for i in range(np.shape(src)[0]):
    #     for j in range(np.shape(src)[1]):
    #         b, g, r = src[i][j]
    #         if r > 95 and g > 40 and b > 20 \
    #             and (max(r, g, b) - min(r, g, b) > 15) \
    #             and abs(r - g) > 15 and r > g and r > b:
    #             dst[i][j] = 255
    return dst

def my_frame_differencing(prev, curr):
    '''
    Function that does frame differencing between the current frame and the previous frame
    Args:
        src The current color image
        prev The previous color image
    Returns:
        dst The destination grayscale image where pixels are colored white if the corresponding pixel intensities in the current
    and previous image are not the same
    '''
    dst = cv2.absdiff(prev, curr)
    gs = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
    dst = (gs > 50).astype(np.uint8) * 255
    return dst

def my_motion_energy(mh):
    '''
    Function that accumulates the frame differences for a certain number of pairs of frames
    Args:
        mh Vector of frame difference images
    Returns:
        dst The destination grayscale image to store the accumulation of the frame difference images
    '''
    dst = np.zeros(np.shape(mh[0]), dtype=np.uint8)
    mask = np.logical_or.reduce([m == 255 for m in mh])
    dst[mask] = 255
    return dst

In [3]:
# ----------------
# a) Reading a stream of images from a webcamera, and displaying the video
# ----------------
# For more information on reading and writing video: http://docs.opencv.org/modules/highgui/doc/reading_and_writing_images_and_video.html
# open the video camera no. 0
cap = cv2.VideoCapture(0)

# if not successful, exit program
if not cap.isOpened():
    print("Cannot open the video cam")
    sys.exit()

# create a window called "MyVideo0"
cv2.namedWindow("MyVideo0", cv2.WINDOW_AUTOSIZE)

# read a new frame from video
ret, frame0 = cap.read()
if not ret:
    print("Cannot read a frame from video stream")

# show the frame in "MyVideo" window
cv2.imshow("MyVideo0", frame0)

# create windows
cv2.namedWindow("FrameDiff", cv2.WINDOW_AUTOSIZE)
cv2.namedWindow("MotionEnergy", cv2.WINDOW_AUTOSIZE)
cv2.namedWindow("Skin", cv2.WINDOW_AUTOSIZE)

len_history = 7
my_motion_history = []
fMH1 = np.zeros(np.shape(frame0)[:-1], dtype=np.uint8)
for i in range(len_history):
    my_motion_history.append(fMH1)


while(1):
    # read a new frame from video
    ret, frame = cap.read()
    # if not successful, break loop
    if not ret:
        print("Cannot read a frame from video stream")
        break

    cv2.imshow("MyVideo0", frame)

    # ----------------
    # b) Skin color detection
    # ----------------
    frame_dst = my_skin_detect(frame)
    cv2.imshow("Skin", frame_dst)

    # ----------------
    # c) Background differencing and motion energy
    # ----------------

    # call my_frame_differencing function
    frame_dst = my_frame_differencing(frame0, frame)
    cv2.imshow("FrameDiff", frame_dst)

    my_motion_history.pop(0)
    my_motion_history.append(frame_dst)

    # ----------------
    #  d) Visualizing motion history
    # ----------------

    # call my_motion_energy function
    myMH = my_motion_energy(my_motion_history)
    cv2.imshow("MotionEnergy", myMH) # show the frame in "MyVideo" window
    frame0 = frame

    # wait for 'esc' key press for 30ms. If 'esc' key is pressed, break loop
    if cv2.waitKey(30) == 27:
        print("esc key is pressed by user")
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

esc key is pressed by user


-1